[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/10--sql-database/01--postgresql.ipynb)

In [ ]:
!pip install psycopg2

In [ ]:
import psycopg2

In [ ]:
connection = psycopg2.connect(user="datascience",
                                  password="datascience",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="datascience")

In [ ]:
# Print PostgreSQL version
cursor = connection.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()

In [ ]:
print(record)

In [ ]:
create_table = '''
DROP TABLE Employee;
CREATE TABLE IF NOT EXISTS Employee (
  id SERIAL PRIMARY KEY,
  name text NOT NULL,
  sallary money NOT NULL
)
'''

In [ ]:
def connect_db():
    connection = psycopg2.connect(user="datascience",
                                  password="datascience",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="datascience")
    return connection

In [ ]:
try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(create_table)
    connection.commit()
except (Exception, psycopg2.DataError) as error:
    print ("Error while creating PostgreSQL table", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
insert_sql = '''
INSERT INTO Employee (name, sallary) VALUES (%s,%s)
''';

try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(insert_sql, ("jakub", 2500.00))
    connection.commit()
except (Exception, psycopg2.DataError) as error:
    print ("Error while inserting", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
select = '''
    select * from Employee;
'''

try:
    connection = connect_db()
    cursor = connection.cursor()
    cursor.execute(select)
    records = cursor.fetchall()
except (Exception, psycopg2.DataError) as error:
    print ("Error while inserting", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
print(records)

## Play with events sale system

```python
def connect_db():
    connection = psycopg2.connect(user="datascience",
                                  password="datascience",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="warehouse")
    return connection
```

```
http://{virtual_machine_ip}:8080
user: datascience
password: datascience
database: warehouse
```
